# Triangle Counting

In this notebook, we will count the numner of trianges in our test dataset.  The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

Notebook Credits
* Original Authors: Bradley Rees
* Created:   08/01/2019
* Last Edit: 08/16/2020

RAPIDS Versions: 0.13

Test Hardware

* GV100 32G, CUDA 10.2


## Introduction
Triangle Counting, as the name implies, finds the number of triangles in a graph.  Triangles are important in computing the clustering Coefficient and can be used for clustering.  


To compute the Pagerank scores for a graph in cuGraph we use:<br>

**cugraph.triangles(G)**
* __G__: cugraph.Graph object


   Compute the triangle (number of cycles of length three) count of the input graph.

    Parameters
    ----------
    G : cugraph.graph
        cuGraph graph descriptor, should contain the connectivity information,
        (edge weights are not used in this algorithm)

    Returns
    -------
    count : int64
        A 64 bit integer whose value gives the number of triangles in the
        graph.

__Reference__:
    O. Green, P. Yalamanchili, L.M. Munguia,
    “Fast Triangle Counting on GPU”
    Irregular Applications: Architectures and Algorithms (IA3), 2014

#### Some notes about vertex IDs...
* The current version of cuGraph requires that vertex IDs be representable as 32-bit integers, meaning graphs currently can contain at most 2^32 unique vertex IDs. However, this limitation is being actively addressed and a version of cuGraph that accommodates more than 2^32 vertices will be available in the near future.
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [ ]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [ ]:
# NetworkX libraries
import networkx as nx
from scipy.io import mmread

### Some Prep

In [ ]:
# Define the path to the test data  
datafile='../data/karate-data.csv'

---
# NetworkX

In [ ]:
# Read the data, this also created a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [ ]:
nx_count = nx.triangles(Gnx)

In [ ]:
# NetworkX does not give a single count, but list how many triangles each vertex is associated with
nx_count

In [ ]:
# To get the number of triangles, we would need to loop through the array and add up each count
count = 0
for key, value in nx_count.items():
    count = count + value

In [ ]:
count

Let's seet how that compares to cuGraph

----

# cuGraph

### Read in the data - GPU
cuGraph depends on cuDF for data loading and the initial Dataframe creation

The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [ ]:
# Test file    
gdf = cudf.read_csv(datafile, delimiter='\t', names=['src', 'dst'], dtype=['int32', 'int32'] )

### Create a Graph 

In [ ]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src', destination='dst')

### Call the Triangle Counting 

In [ ]:
# Call cugraph.pagerank to get the pagerank scores
cu_count = cugraph.triangles(G)

In [ ]:
cu_count

_It was that easy!_  

----


___
Copyright (c) 2019-2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___